<a href="https://colab.research.google.com/github/svandenborne/cheatsheets/blob/master/LLM/zephyr/Information_extraction_with_Zephyr_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Information extraction with Zephyr 7B alpha(α)

Owner Name: Chetan Khadke / Shubham Sanjay Salunke

Email ID: khadkechetan@gmail.com / shubhamss8087@gmail.com

#### 1. Install the packages

In [1]:
!pip install -q pypdf python-dotenv transformers llama-index sentence-transformers langchain
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install  llama-cpp-python --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.9/224.9 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 82.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 7.1 MB/

#### 2. Import the packages

In [2]:
import logging
import sys
import torch
from pprint import pprint
from llama_index.llms import LlamaCPP
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
from llama_index.embeddings import LangchainEmbedding
from llama_index import ServiceContext, set_global_service_context

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

#### 3. Load the dataset

In [3]:
data_location = "/content/data" #@param
documents = SimpleDirectoryReader(data_location).load_data()

ValueError: Directory /content/data does not exist.

#### 4. Load Model

In [ ]:
llm = LlamaCPP(
    model_url='https://huggingface.co/TheBloke/zephyr-7B-alpha-GGUF/resolve/main/zephyr-7b-alpha.Q5_K_M.gguf',
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    context_window=3900,
    generate_kwargs={},
    model_kwargs={"n_gpu_layers": -1},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


#### 5. Load the embeddings

In [ ]:
embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="thenlper/gte-large")
)

In [ ]:
service_context = ServiceContext.from_defaults(
    chunk_size=256,
    llm=llm,
    embed_model=embed_model
)

In [ ]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [ ]:
user_queries = ['Seller address in the document?', 'Client address in the document?', 'seller Tax Id in the document?' ] #@param
answer = dict()
for i, user_query in enumerate(user_queries):
    query_engine = index.as_query_engine()
    response = query_engine.query(user_query)
    answer.update({user_query: response.response})
pprint(answer)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


{'Client address in the document?': '\n'
                                    "Yes, the client's address is provided in "
                                    'the given context information as '
                                    '"Keller-Crosby 280 Kim Valleys Suite 217 '
                                    'Angelaburgh, DE 97356".',
 'Seller address in the document?': '\n'
                                    "Yes, the seller's address is provided in "
                                    'the given context information as '
                                    '"Campbell, Callahan and Gomez 2969 Todd '
                                    'Orchard Apt. 721 Port James, FL 83598".',
 'seller Tax Id in the document?': '\n'
                                   "Yes, the seller's Tax Id is provided in "
                                   'the given context information as '
                                   '"958-83-8233".'}
